<a href="https://colab.research.google.com/github/sheikmohdimran/Experiments_2020/blob/master/NLP/Quora_QA_Pair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
os.environ['KAGGLE_USERNAME'] = "imrandude" # username from the json file
os.environ['KAGGLE_KEY'] = "0bc5f9f7194b6f77f8990f1d092e3af2" # key from the json file
!kaggle competitions download -c quora-question-pairs

 94% 20.0M/21.2M [00:00<00:00, 51.2MB/s]
100% 21.2M/21.2M [00:00<00:00, 84.0MB/s]
  0% 0.00/4.95M [00:00<?, ?B/s]
100% 4.95M/4.95M [00:00<00:00, 80.8MB/s]
 97% 168M/173M [00:01<00:00, 142MB/s]
100% 173M/173M [00:01<00:00, 143MB/s]
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
!ls

sample_data  sample_submission.csv.zip	test.csv.zip  train.csv.zip


In [22]:
!unzip train.csv.zip
!unzip test.csv.zip

Archive:  test.csv.zip
  inflating: test.csv                


In [0]:
!mkdir ./custom_embedding

In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from datetime import datetime
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
print("Start Time {}".format(datetime.now()))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Start Time 2020-02-06 13:47:17.055014


In [0]:
pre_trained_model = None
glove_embedding = 'glove_embedding'
glove_path = 'glove.6B.300d.txt'
fasttext_embedding = 'fasttext_embedding'
fasttext_path = 'wiki-news-300d-1M.vec'
pre_train_model_dimension = 300
embedding_dim = 300


In [0]:
def generate_glove_dict(path):
    glove_index = dict()
    gl_path = os.path.join('pre-trained-model', path)

    file = open(gl_path, encoding="utf8")

    for line in file:
        splited_line = line.split()
        word = splited_line[0]
        coeficient = np.array(splited_line[1:], dtype='float32')

        glove_index[word] = coeficient

    file.close()
    return glove_index



def get_embedding_matrix(glove_dict, token_items, vocab_size, vector_dim):
    marix = np.zeros((vocab_size, vector_dim))

    for word, idx in token_items:
        #word = lemmatizer.lemmatize(word.lower().strip())
        vector = glove_dict.get(word)
        if vector is not None:
            marix[idx] = vector

    return marix



def get_glove_matrix(glove_path, token_items, vocab_size, dimension):
    glove_dict = generate_glove_dict(glove_path)
    embed_matrix = get_embedding_matrix(glove_dict, token_items, vocab_size, dimension)
    return embed_matrix


def get_fasttext_matrix(glove_path, token_items, vocab_size, dimension):
    glove_dict = generate_glove_dict(glove_path)
    embed_matrix = get_embedding_matrix(glove_dict, token_items, vocab_size, dimension)
    return embed_matrix

def save_tokenizer(tokenizer, model_folder):
    with open(os.path.join(model_folder,'tokenizer_200_b.pickle'), 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

def classification_report1(test_data, test_labels):
    test_labels = np.array(test_label)
    question1 = test_data['question1']
    question2 = test_data['question2']
    sequences1 = loaded_tokenizer.texts_to_sequences(question1)
    sequences2 = loaded_tokenizer.texts_to_sequences(question2)
    padded_sequences1 = pad_sequences(sequences1, maxlen =20, padding='post')
    padded_sequences2 = pad_sequences(sequences2, maxlen =20, padding='post')
    test_predicted = model.predict([padded_sequences1, padded_sequences2])
    predicted_labels = np.round(test_predicted)
    print(predicted_labels)
    predicted_labels = [ int(label[0]) for label in predicted_labels]
    print("Classification report ========> ")
    print(classification_report(test_labels, predicted_labels))
    print("Accuracy score ======> ")
    print(accuracy_score(test_labels, predicted_labels))
    return True


In [0]:
# Add tf.keras.layers.InputLayer --> 
# https://stackoverflow.com/questions/45217973/what-is-the-advantage-of-using-an-inputlayer-or-an-input-in-a-keras-model-with

def get_model(vocab_size, embedding_dim, embedding_matrix, input_length):
    model = tf.keras.Sequential([
	  tf.keras.layers.InputLayer(input_shape=input_length*2),
    tf.keras.layers.Embedding(input_dim = vocab_size, output_dim = embedding_dim, weights=[embedding_matrix],input_length = input_length*2,trainable=False),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(30)),
    tf.keras.layers.Dense(30,activation = 'relu'),
    tf.keras.layers.Dense(20,activation = 'relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

    return model

In [0]:
df=pd.read_csv('train.csv',skiprows=1,header=None)


In [0]:
train_df=df.iloc[:,3:5].values
train_labels=df.iloc[:,5:6].values
epochs = 5
batch_size = 1000
maxlen = 20
input_length = 20
n_split=2
i=1


In [33]:
for train_index,test_index in KFold(n_split).split(train_df):
  print("Preprocessing + Training + Plotting + Test data preprocessing + Evaluation + Prediction + Reporting Start Time {}".format(datetime.now()))
  print("Pre-Processing Start Time {}".format(datetime.now()))
  x_train,x_test=train_df[train_index],train_df[test_index]
  y_train,y_test=train_labels[train_index],train_labels[test_index]


  question1, question2 = [], []
  for each in x_train:#['question1'].to_string(index=False)
      question1.append(str(each[0]))
      question2.append(str(each[1]))

  tokenizer = Tokenizer(oov_token='<OOV>')
  tokenizer.fit_on_texts(question1+question2)
  vocab_size = len(tokenizer.word_index)+1

  sequences1 = tokenizer.texts_to_sequences(question1)
  sequences2 = tokenizer.texts_to_sequences(question2)

  padded_sequences1 = pad_sequences(sequences1, maxlen =maxlen, padding='post')
  padded_sequences2 = pad_sequences(sequences2, maxlen =maxlen, padding='post')
  padded_sequences = np.concatenate((padded_sequences1, padded_sequences2),axis=1)

  if pre_trained_model == 'glove':
      model_folder = glove_embedding
      embedding_matrix = get_glove_matrix(glove_path, tokenizer.word_index.items(), vocab_size, pre_train_model_dimension)
  elif pre_trained_model == 'fasttext':
      model_folder = fasttext_embedding
      embedding_matrix = get_fasttext_matrix(fasttext_path, tokenizer.word_index.items(), vocab_size, pre_train_model_dimension)
  else:
      model_folder = './custom_embedding'
      embedding_matrix = np.zeros((vocab_size, embedding_dim))

  with open(os.path.join(model_folder,'tokenizer_200_b_kfold_'+str(i)+'.pickle'), 'wb') as handle:
      pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

  print("Pre-Processing End Time {}".format(datetime.now()))
  print("Model Training Start Time {}".format(datetime.now()))
  model=get_model(vocab_size, embedding_dim, embedding_matrix, input_length)
  model.compile(loss='binary_crossentropy', optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001,decay = 1e-6, momentum=0.9), metrics=['accuracy'])
  model.summary()

  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
  # This callback will stop the training when there is no improvement in
  #the validation loss for three consecutive epochs.

  history = model.fit(padded_sequences,#[padded_sequences1, padded_sequences2], 
                      y_train, 
                      callbacks=[callback], batch_size=batch_size, 
                      epochs = epochs, validation_split=0.2)
  #history = model.fit([padded_sequences1, padded_sequences2], y_train, epochs = epochs, steps_per_epoch=5, batch_size=batch_size)
  print("Model Training End Time {}".format(datetime.now()))
  model.save(os.path.join(model_folder,'model_200_b_kfold_'+str(i)+'.h5'))

  i +=1
  print("Model Created")
  print("<==============================================================>")
  print("Prediction/Evaluation Started")
  print("Test Data Pre-processing Start Time {}".format(datetime.now()))
  question1, question2 = [], []
  for each in x_test:#['question1'].to_string(index=False)
      question1.append(str(each[0]))
      question2.append(str(each[1]))
  
  sequences1 = tokenizer.texts_to_sequences(question1)
  sequences2 = tokenizer.texts_to_sequences(question2)

  padded_sequences1 = pad_sequences(sequences1, maxlen =20, padding='post')
  padded_sequences2 = pad_sequences(sequences2, maxlen =20, padding='post')
  padded_sequences = np.concatenate((padded_sequences1, padded_sequences2),axis=1)

  print("Test Data Pre-processing End Time {}".format(datetime.now()))
  #print('Model evaluation ',model.evaluate(padded_sequences,y_test))
  
  print("Test Data Prediction Start Time {}".format(datetime.now()))
  #y_test_predicted = model.predict([padded_sequences1, padded_sequences2], steps=5)
  y_test_predicted = model.predict(padded_sequences)#, steps=5)
  print("Test Data Prediction End Time {}".format(datetime.now()))

  print("Preprocessing + Training + Plotting + Test data preprocessing + Evaluation + Prediction + Reporting End Time {}".format(datetime.now()))
  print("Prediction/Evaluation End")
  print("<==================================================================================================================>")
print("End Time {}".format(datetime.now()))



Preprocessing + Training + Plotting + Test data preprocessing + Evaluation + Prediction + Reporting Start Time 2020-02-06 14:30:37.708317
Pre-Processing Start Time 2020-02-06 14:30:37.708811
Pre-Processing End Time 2020-02-06 14:30:55.682226
Model Training Start Time 2020-02-06 14:30:55.682421
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 40, 300)           20874600  
_________________________________________________________________
bidirectional_6 (Bidirection (None, 60)                79440     
_________________________________________________________________
dense_18 (Dense)             (None, 30)                1830      
_________________________________________________________________
dense_19 (Dense)             (None, 20)                620       
_________________________________________________________________
dense_20 (Dense)       

## Inference Code

In [1]:
from tensorflow.keras import backend as K
K.set_learning_phase(0)
from tensorflow.keras.models import load_model
model = load_model('./custom_embedding/model_200_b_kfold_1.h5')
print(model.inputs)
print(model.outputs)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
[<tf.Tensor 'embedding_6_input:0' shape=(?, 40) dtype=float32>]
[<tf.Tensor 'dense_20/Sigmoid:0' shape=(?, 1) dtype=float32>]


In [2]:
from tensorflow.keras import backend as K
import tensorflow as tf

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        print(tf.global_variables())
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),output_names=[out.op.name for out in model.outputs])


[<tf.Variable 'embedding_6/embeddings:0' shape=(69582, 300) dtype=float32>, <tf.Variable 'bidirectional_6/forward_lstm_6/kernel:0' shape=(300, 120) dtype=float32>, <tf.Variable 'bidirectional_6/forward_lstm_6/recurrent_kernel:0' shape=(30, 120) dtype=float32>, <tf.Variable 'bidirectional_6/forward_lstm_6/bias:0' shape=(120,) dtype=float32>, <tf.Variable 'bidirectional_6/backward_lstm_6/kernel:0' shape=(300, 120) dtype=float32>, <tf.Variable 'bidirectional_6/backward_lstm_6/recurrent_kernel:0' shape=(30, 120) dtype=float32>, <tf.Variable 'bidirectional_6/backward_lstm_6/bias:0' shape=(120,) dtype=float32>, <tf.Variable 'dense_18/kernel:0' shape=(60, 30) dtype=float32>, <tf.Variable 'dense_18/bias:0' shape=(30,) dtype=float32>, <tf.Variable 'dense_19/kernel:0' shape=(30, 20) dtype=float32>, <tf.Variable 'dense_19/bias:0' shape=(20,) dtype=float32>, <tf.Variable 'dense_20/kernel:0' shape=(20, 1) dtype=float32>, <tf.Variable 'dense_20/bias:0' shape=(1,) dtype=float32>, <tf.Variable 'traini

In [3]:
tf.train.write_graph(frozen_graph, "./custom_embedding/", "model_200_b_kfold_1.pb", as_text=False)

'./custom_embedding/model_200_b_kfold_1.pb'

In [4]:
import tensorflow as tf
from tensorflow.python.platform import gfile

f = gfile.FastGFile("./custom_embedding/model_200_b_kfold_1.pb", 'rb')
graph_def = tf.GraphDef()
graph_def.ParseFromString(f.read())
f.close()

sess = K.get_session()
tf.import_graph_def(graph_def)


Instructions for updating:
Use tf.gfile.GFile.


In [5]:
import os
import numpy as np
from datetime import datetime
import pickle
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

with open(os.path.join('./custom_embedding', 'tokenizer_200_b_kfold_1.pickle'), 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)

df = pd.read_csv('./test.csv')
df = df.dropna()


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
def get_microseconds(s_date, e_date):
    diff = e_date-s_date
    microseconds = 1000000 * diff.seconds
    microseconds = microseconds + diff.microseconds
    return microseconds

def microsecond_to_seconds(microseconds):
    seconds = microseconds / 1000000
    return seconds

sigmoid_tensor = sess.graph.get_tensor_by_name('import/dense_20/Sigmoid:0')
processing_records = 200000
inference_time = 0
def inference_report(test_data):
    question1 = test_data['question1']
    question2 = test_data['question2']
    sequences1 = loaded_tokenizer.texts_to_sequences(question1)
    sequences2 = loaded_tokenizer.texts_to_sequences(question2)
    padded_sequences1 = pad_sequences(sequences1, maxlen =20, padding='post')
    padded_sequences2 = pad_sequences(sequences2, maxlen =20, padding='post')
    padded_sequences = np.concatenate((padded_sequences1, padded_sequences2), axis=1)
    #padded_sequences1 = tf.convert_to_tensor(padded_sequences1)
    #padded_sequences2 = tf.convert_to_tensor(padded_sequences2)    
    predict = model.predict(padded_sequences)#[padded_sequences1, padded_sequences2], steps=5)
    print(predict)
    #aa = np.concatenate((tf.convert_to_tensor(padded_sequences1), tf.convert_to_tensor(padded_sequences2)), axis=None)
    start_inf = datetime.now()
    predictions = sess.run(sigmoid_tensor, {'import/embedding_6_input:0': np.concatenate((padded_sequences1, padded_sequences2), axis=1)})
    #predictions = sess.run(sigmoid_tensor, {'import/embedding_input:0': [padded_sequences1, padded_sequences2]})
    end_inf = datetime.now()
    inference_time = get_microseconds(start_inf, end_inf)
    
    print("Total Inference Time {}(microseconds) for {} Records".format(inference_time, processing_records))
    avg_inf_time = float(microsecond_to_seconds(inference_time)/processing_records)
    print("Average Inference Time {} for One record".format(avg_inf_time))
    
inference_report(df[0:processing_records])    


[[0.36885345]
 [0.36885345]
 [0.36885345]
 ...
 [0.36885345]
 [0.36885345]
 [0.36885345]]
